In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


# Data files full clean

In [3]:
import os
import re
if not os.path.exists('./cleandata'):
    os.makedirs('./cleandata')
content=os.listdir('./newdata')

for item in content:
    # delete the text files and the urls files to clean the library
    print(item)
    if (item.find(".txt") > 0 ):
        #print(item)
        with open("./newdata/{}".format(item),"r", encoding='iso-8859-1') as f:
            lines=f.readlines()
        #if (os.path.exists("./cleandata/{}".format(item))):
        with open("./cleandata/{}".format(item),"w+") as f:
             
            for line in lines:
                if "10 Messungen" in line.strip("\n"):
                    line= line[line.find("Ch"):]
                    #print(newline)
                # if (line.strip("\n") != "þStartfischer1 p33-6 hf8 fc6x 0.9km")
                if ("Ch" in line.strip("\n") or "V" in line.strip("\n") ):
                    f.write(line.replace(';',''))
f.close();

#print(content)

fischer2_p33_6_hf8_fc6x_4km.txt
madshus_base7_ts_blue_ clockwise 45 turn.txt
fischer1_p33_6_hf8_fc6x_4km.txt
madshus_base7_ts_blue_point.txt
madshus_base7_ts_blue.txt
fischer_p33_6_hf8_fc6x.txt
fischer2_p33_6_hf8_fc6x.txt
fischer1_p33_6_hf8_fc6x_0_9km.txt
fischer2_p33_6_hf8_fc6x_0_9km.txt


In [ ]:
a={}

for x in content:
    print(x)
    df=pd.read_csv("./newdata/{}".format(x),names=['channal','min','max','x','y'],sep='\s+',engine='python', encoding='iso-8859-1')
    dx=df.drop(['x', 'y'], axis=1)
    print(df.head())
    # dx['min']= dx['min'].map(lambda x: x.lstrip('+-;').rstrip('aAbBcC;'))
    # dx['channal']= dx['channal'].map(lambda x: x.lstrip(';').rstrip(';'))
    # #idx=dx.index();
    # if 100 in dx.index:
    #     dx=dx.drop(100);
    # dx['min']=pd.to_numeric(dx['min'])
    # dx['max']=pd.to_numeric(dx['max'])
    # dx.index.name=x
    # a[x]=dx
    # print(a[x].head())
    # break;




In [ ]:
# df=pd.read_csv("./newdata/fischer1_p33_6_hf8_fc6x_4km.txt",names=['channal','min','max','x','y'],sep='\s+',engine='python',encoding='iso-8859-1',skiprows=1)
# dx=df.drop(['x', 'y'], axis=1)
# dx['min']= dx['min'].map(lambda x: x.lstrip('+-;').rstrip('aAbBcC;'))
# dx['channal']= dx['channal'].map(lambda x: x.lstrip(';').rstrip(';'))
# #dx=dx.drop(100);
# dx['min']=pd.to_numeric(dx['min'])
# dx['max']=pd.to_numeric(dx['max'])
# dx.index.name="fischer1_p33_6_hf8_fc6x_4km"
# dx.to_excel("formatet.xlsx");
# dx.head(10)

In [5]:
print("hallo Wolrd!")

hallo Wolrd!
